In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import GPy
import numpy as np
from GPy.core import GP
from GPy import likelihoods
from GPy import kern
from GPy import util
import numpy as np
import pylab as pb
from sklearn.metrics import mean_absolute_error, mean_squared_error
import itertools
from cwgp.grid_search import grid_search
import seaborn as sns
import copy
import os
import scipy
from sklearn.model_selection import KFold

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
class GPCoregionalizedRegression(GP):
    """
    Gaussian Process model for heteroscedastic multioutput regression

    This is a thin wrapper around the models.GP class, with a set of sensible defaults

    :param X_list: list of input observations corresponding to each output
    :type X_list: list of numpy arrays
    :param Y_list: list of observed values related to the different noise models
    :type Y_list: list of numpy arrays
    :param kernel: a GPy kernel ** Coregionalized, defaults to RBF ** Coregionalized
    :type kernel: None | GPy.kernel defaults
    :likelihoods_list: a list of likelihoods, defaults to list of Gaussian likelihoods
    :type likelihoods_list: None | a list GPy.likelihoods
    :param name: model name
    :type name: string
    :param W_rank: number tuples of the corregionalization parameters 'W' (see coregionalize kernel documentation)
    :type W_rank: integer
    :param kernel_name: name of the kernel
    :type kernel_name: string
    """
    def __init__(self, X_list, Y_list, kernel=None, mean_function=None, likelihoods_list=None, name='GPCR',W_rank=1,kernel_name='coreg'):

        #Input and Output
        X,Y,self.output_index = util.multioutput.build_XY(X_list,Y_list)
        Ny = len(Y_list)

        #Kernel
        if kernel is None:
            kernel = kern.RBF(X.shape[1]-1)
            
            kernel = util.multioutput.ICM(input_dim=X.shape[1]-1, num_outputs=Ny, kernel=kernel, W_rank=W_rank,name=kernel_name)

        #Likelihood
        likelihood = util.multioutput.build_likelihood(Y_list,self.output_index,likelihoods_list)

        super(GPCoregionalizedRegression, self).__init__(X,Y,kernel,likelihood,mean_function=mean_function, Y_metadata={'output_index':self.output_index})

In [3]:
def const(x):
    if x.shape[0]!=80:
        print(x)
    return x

In [4]:
def plot_2outputs(m,x,Y_train,X_test,Y_test,xlim,ylim):
    fig = pb.figure(figsize=(12,8))
    #Output 1
    ax1 = fig.add_subplot(211)
    ax1.set_xlim(xlim)
    ax1.set_title('Output 1')
    m.plot(plot_limits=xlim,fixed_inputs=[(1,0)],which_data_rows=slice(0,len(Y_train[0])),ax=ax1)
    ax1.plot(x,Y_train[0],'x',color = "black",mew=1.5)
    #Output 2
    ax2 = fig.add_subplot(212)
    ax2.set_xlim(xlim)
    ax2.set_title('Output 2')
    m.plot(plot_limits=xlim,fixed_inputs=[(1,1)],which_data_rows=slice(len(Y_train[0]),2*len(Y_train[0])),ax=ax2)
    ax2.plot(x,Y_train[1],'x',color = "black",mew=1.5) 
    if X_test is not None :
        ax1.plot(X_test,Y_test[0],'x',color = "red",mew=1.5)
        ax2.plot(X_test,Y_test[1],'x',color = "red",mew=1.5)
    plt.show()

In [5]:
def draw(domain, y_mean, y_top, y_bot, x_train, y_train, x_val, y_val, x_test, y_test, subplot, ylim=False):
    subplot.fill_between(domain, np.ravel(y_top), np.ravel(y_bot), color=(0,0.5,0.5,0.2), label="Confidence")
    subplot.scatter(x_train, y_train, marker="x", color='black', label="train")
    subplot.scatter(x_val, y_val, marker="x", color='red', label="validate")
    subplot.scatter(x_test, y_test, marker="x", color='blue', label="test")
    subplot.plot(domain, y_mean, label="mean")
    if ylim:
        subplot.set_ylim(ylim)       
    subplot.grid(True)
    subplot.legend()

In [6]:
def mogp_cwgp(**kwargs):
    
    cwgp_models = kwargs["model_holder"]
    train = kwargs["train"]
    val = kwargs["val"]
    X,Y = kwargs["X"], kwargs["Y"]
    x_test, y_test = kwargs["x_test"], kwargs["y_test"]
    mean_function = kwargs["mean_function"]

    
    
    X_train, Y_train = X[train], Y[:,train,:]
    X_val, Y_val = X[val], Y[:,val,:]
    
    

    X_jp_train, Y_jp_train = X_train, Y_train[0]
    X_us_train, Y_us_train = X_train, Y_train[1]
    x_jp_val, y_jp_val = X_val, Y_val[0]
    x_us_val, y_us_val = X_val, Y_val[1]
    X_jp_test, Y_jp_test = x_test, y_test[0]
    X_us_test, Y_us_test = x_test, y_test[1]
    

    
    for cwgp in cwgp_models:
        Y_jp_train, d = cwgp.phi.comp_phi(cwgp.phi.res.x, Y_jp_train)
        Y_us_train, d = cwgp.phi.comp_phi(cwgp.phi.res.x, Y_us_train)
        Y_train, d = cwgp.phi.comp_phi(cwgp.phi.res.x, Y_train)
        Y_val, d = cwgp.phi.comp_phi(cwgp.phi.res.x, Y_val)
    


    mf = GPy.core.Mapping(1,1)
    mf.f = lambda x: mean_function(x)
    mf.update_gradients = lambda a,b: None
     
    
    K = GPy.kern.RBF(1)
    icm = GPy.util.multioutput.ICM(input_dim=1,num_outputs=2,kernel=K)
    
    
    n = GPCoregionalizedRegression([X_jp_train, X_us_train],[Y_jp_train,Y_us_train],kernel=icm,mean_function=mf)
    n.optimize()
    
    start, end = 1940, 2031
        
    plot_2outputs(n,X_train,Y_train,X_val,Y_val,xlim=(start-10,end+10),ylim=(0.01,0.08))
   
    X_jp_val = np.hstack([X_val,np.zeros_like(X_val)])
    noise_dict = {'output_index':X_jp_val[:,1:].astype(int)}
    Y_jp_val = n.predict(X_jp_val, Y_metadata=noise_dict)
    Y_jp_val_mean, Y_jp_val_var=Y_jp_val[0],Y_jp_val[1]


    for cwgp in cwgp_models[::-1]:
        Y_jp_val_mean = cwgp.phi.inv_comp_phi(cwgp.phi.res.x, Y_jp_val_mean)
        Y_train = cwgp.phi.inv_comp_phi(cwgp.phi.res.x, Y_train)
        Y_val = cwgp.phi.inv_comp_phi(cwgp.phi.res.x, Y_val)

    x_jp_test = np.hstack([X_jp_test,np.zeros_like(X_jp_test)])
    
    
    domain = np.linspace(start,end,end-start+1).reshape(-1,1)
    domain_2 = np.hstack([domain,np.zeros_like(domain)])
    domain_3 = np.linspace(start,end,end-start+1)
    noise_dict = {'output_index':domain_2[:,1:].astype(int)}
    cwgp_y_all = n.predict(domain_2,Y_metadata=noise_dict)
    cwgp_y_mean, cwgp_y_var = cwgp_y_all[0], cwgp_y_all[1]
    cwgp_y_top, cwgp_y_bot = cwgp_y_mean + 1.96*np.sqrt(cwgp_y_var), cwgp_y_mean - 1.96*np.sqrt(cwgp_y_var) 
    noise_dict2 = {'output_index':x_jp_test[:,1:].astype(int)}
    cwgp_predict_y_test = n.predict(x_jp_test,Y_metadata=noise_dict2)
    cwgp_predict_y_test_mean, cwgp_predict_y_test_var = cwgp_predict_y_test[0],cwgp_predict_y_test[1]
    
    y_mean, y_var, y_top, y_bot ,predict_y_test_mean = cwgp_y_mean, cwgp_y_var, cwgp_y_top, cwgp_y_bot, cwgp_predict_y_test_mean
    
    for cwgp in cwgp_models[::-1]: 
        y_mean, y_var = cwgp.phi.inv_comp_phi(cwgp.phi.res.x, y_mean), cwgp.phi.inv_comp_phi(cwgp.phi.res.x, y_var)
        y_top, y_bot = cwgp.phi.inv_comp_phi(cwgp.phi.res.x, y_top), cwgp.phi.inv_comp_phi(cwgp.phi.res.x, y_bot)
        predict_y_test_mean = cwgp.phi.inv_comp_phi(cwgp.phi.res.x, predict_y_test_mean)
    
    validate_rmse = mean_squared_error(Y_jp_val_mean, 
                                       y_jp_val,
                                       squared=False)
    
    test_rmse = mean_squared_error(predict_y_test_mean, 
                                   Y_jp_test,
                                   squared=False)
    print(validate_rmse,test_rmse)
    
        
    fig, (ax1, ax2) = plt.subplots(2,1,figsize=(10,6))
    
    draw(domain_3, y_mean, y_top, y_bot, X_jp_train, Y_train[0], X_val, 
        Y_jp_val_mean, x_test, y_test[0], ax1)

    print(kwargs["hyperparams"])
    print(
    f"validate_rmse: {validate_rmse} \n test_rmse: {test_rmse}\n"
    )
    plt.show()
    return validate_rmse, test_rmse
#     print ('\nB matrix\n',icm.B.B)
#     print (n)
    

In [7]:
## Preprocess

file_names = ["JP","USA"]
age_range = [69,70]


df = {}
df_age = {}


for country in file_names:
    df[country] = pd.read_csv(f"{country}.txt", skiprows=1, delim_whitespace=True)
    df[country] = df[country][df[country]["Age"] != "110+"]
    df[country]["Total"] = pd.to_numeric(df[country]["Total"],errors='coerce')
    df[country]["Age"] = df[country]["Age"].astype("int64")


    df[country]["country"] = country
    
blank_df = pd.DataFrame(columns=df[file_names[0]].columns)

for age in age_range:
    df_age[age] = blank_df
    for country in file_names:
        df_age[age] = pd.concat([df_age[age],df[country][df[country]["Age"] == age]])

In [8]:
age = 70
start, end = 1947,2019

In [9]:
X = np.arange(start,end).reshape(-1,1)
Y1 = df_age[age].query(f"country == 'JP' and  {end} > Year >= {start} ")["Total"].to_numpy().reshape(-1,1)
Y2 = df_age[age].query(f"country == 'USA' and {end} > Year >= {start} ")["Total"].to_numpy().reshape(-1,1)
Y = np.stack([Y1,Y2])

In [10]:
test_ratio = 1 - 0.15
length = len(X)

test = int(test_ratio*length)

x_tv, y_tv = X[:test], Y[:,:test]
x_test, y_test = X[test:], Y[:,test:]

In [11]:
x_tv.shape

(61, 1)

In [12]:
# SOGP

In [13]:
# MOGP

In [14]:
grid_search(mogp_cwgp, x_tv , y_tv[0], {"c":4,"n":[2],"transformations":["box_cox","sa"]}, mean_function=const,n_splits=3,random_state=1,shuffle=True, cv=True, X=X,Y=Y, x_test=x_test, y_test=y_test)

  0%|          | 0/16 [00:00<?, ?it/s]

[('box_cox', 2), ('sa', 2)]


AssertionError: 